In [20]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
from census import Census

# Census API Key
from config import api_key
c = Census(api_key, year=2016)

In [25]:
census_data = c.acs5.get(("NAME", "B19013_001E", "B01003_001E", "B01002_001E","B25064_001E","B15003_022E",
                          "B19301_001E","B25077_001E","B23025_005E","B25035_001E","B15003_023E",
                          "C24010_006E","C24010_007E","C24010_008E","B08136_003E",
                          "C24010_009E","C24010_042E","C24010_043E",
                          "C24010_044E","C24010_045E",
                          "B17001_002E"), {'for': 'zip code tabulation area:*'})
# Convert to DataFrame
census_pd = pd.DataFrame(census_data)

# Column Reordering
census_pd = census_pd.rename(columns={"B01003_001E": "Population",
                                      "B01002_001E": "Median Age",
                                      "B23025_005E":"Unemployment",
                                      "B08136_003E":"commute time car",
                                      "B15003_023E":"Masters holders",
                                      "B15003_022E":"Bachelor holders",
                                      "B25064_001E":"Median gross rent",
                                      "B25035_001E":"House Age",
                                      "B19013_001E": "Household Income",
                                      "B25077_001E":"median_home_value",
                                      "B19301_001E": "Per Capita Income",
                                      "B17001_002E": "Poverty Count",
                                      "C24010_006E":"Men/fin/biz",
                                      "C24010_007E":"Men/stem",
                                      "C24010_008E":"Men/tech",
                                      "C24010_009E":"Men/eng",
                                      "C24010_042E":"wom/fin/biz",
                                      "C24010_043E":"wom/stem",
                                      "C24010_044E":"wom/tech",
                                      "C24010_045E":"wom/eng",
                                      "NAME": "Name", "zip code tabulation area": "Zipcode"})

census_pd.head()

,Median Age,Population,commute time car,Bachelor holders,Masters holders,Poverty Count,Household Income,Per Capita Income,Unemployment,House Age,...,Men/fin/biz,Men/stem,Men/tech,Men/eng,wom/fin/biz,wom/stem,wom/tech,wom/eng,Name,Zipcode
0,45.0,17423.0,NaN,2016.0,1353.0,1462.0,56714.0,30430.0,479.0,1969.0,...,242.0,302.0,170.0,117.0,219.0,109.0,70.0,32.0,ZCTA5 01001,01001
1,23.2,29970.0,188755.0,3161.0,2956.0,8351.0,48923.0,26072.0,1271.0,1971.0,...,221.0,576.0,206.0,168.0,259.0,408.0,103.0,51.0,ZCTA5 01002,01002
2,19.9,11296.0,NaN,3.0,10.0,54.0,2499.0,3829.0,1021.0,1971.0,...,0.0,62.0,52.0,10.0,7.0,82.0,26.0,16.0,ZCTA5 01003,01003
3,44.1,5228.0,NaN,505.0,327.0,230.0,70568.0,32169.0,153.0,1971.0,...,44.0,139.0,19.0,104.0,45.0,0.0,0.0,0.0,ZCTA5 01005,01005
4,42.5,14888.0,NaN,2185.0,1455.0,1410.0,80502.0,36359.0,543.0,1982.0,...,177.0,345.0,171.0,147.0,264.0,57.0,13.0,0.0,ZCTA5 01007,01007


In [26]:
# Add in Poverty Rate (Poverty Count / Population)
census_pd["Poverty Rate"] = 100 * \
    census_pd["Poverty Count"].astype(
        int) / census_pd["Population"].astype(int)

census_pd["Unemployment rate"] = 100 * \
    census_pd["Unemployment"].astype(
        int) / census_pd["Population"].astype(int)
census_pd["house_age"] = 2018 - census_pd['House Age'].astype(int)

census_pd['pop_biz'] = census_pd["Men/fin/biz"].astype(int) + census_pd["wom/fin/biz"].astype(int)
census_pd['pop_tech'] = census_pd["Men/tech"].astype(int) + census_pd["wom/tech"].astype(int)
census_pd['pop_stem'] = census_pd["Men/stem"].astype(int) + census_pd["wom/stem"].astype(int)
census_pd['pop_arc/eng'] = census_pd["Men/eng"].astype(int) + census_pd["wom/eng"].astype(int)


# Final DataFrame
census_pd = census_pd[["Zipcode", "Population", "Median Age", "Household Income","median_home_value",
                       "Per Capita Income","Masters holders",
                       "Bachelor holders","Median gross rent" ,"Poverty Count", 
                       "Poverty Rate","Unemployment","Unemployment rate",
                      'pop_arc/eng','pop_stem','pop_tech','pop_biz',"commute time car","house_age"]]

# Visualize
print(len(census_pd))
census_pd.head()

33120


,Zipcode,Population,Median Age,Household Income,median_home_value,Per Capita Income,Masters holders,Bachelor holders,Median gross rent,Poverty Count,Poverty Rate,Unemployment,Unemployment rate,pop_arc/eng,pop_stem,pop_tech,pop_biz,commute time car,house_age
0,01001,17423.0,45.0,56714.0,202800.0,30430.0,1353.0,2016.0,975.0,1462.0,8.391207,479.0,2.749240,149,411,240,461,NaN,49
1,01002,29970.0,23.2,48923.0,344000.0,26072.0,2956.0,3161.0,1206.0,8351.0,27.864531,1271.0,4.240908,219,984,309,480,188755.0,47
2,01003,11296.0,19.9,2499.0,-666666666.0,3829.0,10.0,3.0,1150.0,54.0,0.478045,1021.0,9.038598,26,144,78,7,NaN,47
3,01005,5228.0,44.1,70568.0,213700.0,32169.0,327.0,505.0,926.0,230.0,4.399388,153.0,2.926549,104,139,19,89,NaN,47
4,01007,14888.0,42.5,80502.0,258000.0,36359.0,1455.0,2185.0,921.0,1410.0,9.470715,543.0,3.647233,147,402,184,441,NaN,36


In [27]:
census_pd.to_csv("census_data2.csv", encoding="utf-8", index=False)

In [11]:
census_pd.describe()

,Population,Median Age,Household Income,median_home_value,Per Capita Income,Masters holders,Bachelor holders,Median gross rent,Poverty Count,Poverty Rate,Unemployment,Unemployment rate,pop_arc/eng,pop_stem,pop_tech,pop_biz
count,33120.000000,3.312000e+04,3.312000e+04,3.312000e+04,3.283600e+04,33120.000000,33120.000000,3.312000e+04,33120.000000,32799.000000,33120.000000,32799.000000,33120.000000,33120.000000,33120.000000,33120.000000
mean,9724.409300,-1.056759e+07,-4.036731e+07,-4.705542e+07,-6.489877e+06,530.292301,1226.267271,-1.156797e+08,1464.617935,14.616273,363.300876,3.383891,82.158243,246.603442,125.043448,217.028140
std,14358.657599,8.326838e+07,1.591133e+08,1.710795e+08,6.559602e+07,1026.797356,2184.776523,2.524686e+08,2844.537901,11.458578,635.292863,3.132990,173.916919,540.660688,313.316914,419.593829
min,0.000000,-6.666667e+08,-6.666667e+08,-6.666667e+08,-6.666667e+08,0.000000,0.000000,-6.666667e+08,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,718.000000,3.640000e+01,3.785700e+04,8.270000e+04,2.039000e+04,16.000000,50.000000,5.250000e+02,79.000000,6.904462,17.000000,1.798190,0.000000,4.000000,0.000000,4.000000
50%,2807.500000,4.160000e+01,4.892900e+04,1.234000e+05,2.510000e+04,86.000000,222.000000,6.920000e+02,337.000000,12.195122,86.000000,2.983178,12.000000,33.000000,11.000000,33.000000
75%,13177.750000,4.670000e+01,6.218800e+04,1.966000e+05,3.135725e+04,540.000000,1341.000000,9.120000e+02,1492.000000,19.405079,428.000000,4.311606,82.000000,223.000000,98.000000,222.000000
max,115104.000000,9.530000e+01,2.500010e+05,2.000001e+06,2.281240e+05,16057.000000,25371.000000,3.501000e+03,40661.000000,100.000000,9698.000000,100.000000,3957.000000,11011.000000,9189.000000,8139.000000
